## 문장의 유사도
- countvectorizer 문장을 벡터로 변환하는 함수

In [1]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=1)
vectorizer

CountVectorizer()

### 훈련용 문장

In [2]:
contents = ['상처받은 아이들은 너무 일찍 커버려',
            '내가 상처받은 거 아는 사람 불편해',
            '잘 사는 사람들은 좋은 사람 되기 쉬워',
            '아무 일도 아니야 괜찮아']

In [3]:
from konlpy.tag import Twitter
t = Twitter()
t

/home/soomin/venv/nltk/lib/python3.10/site-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [4]:
contents_token = []
for row in contents:
    contents_token.append(t.morphs(row))

contents_token

[['상처', '받은', '아이', '들', '은', '너무', '일찍', '커버', '려'],
 ['내', '가', '상처', '받은', '거', '아는', '사람', '불편해'],
 ['잘', '사는', '사람', '들', '은', '좋은', '사람', '되기', '쉬워'],
 ['아무', '일도', '아니야', '괜찮아']]

### 다시 문장으로~
- 형태소 분석된 결과를 다시 하나의 문장씩으로 합친다

In [5]:
contents_for_vectorize = []

for content in contents_token:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word

    contents_for_vectorize.append(sentence)

contents_for_vectorize

[' 상처 받은 아이 들 은 너무 일찍 커버 려',
 ' 내 가 상처 받은 거 아는 사람 불편해',
 ' 잘 사는 사람 들 은 좋은 사람 되기 쉬워',
 ' 아무 일도 아니야 괜찮아']

### vectorize

In [22]:
X = vectorizer.fit_transform(contents_for_vectorize)
X

<4x17 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [23]:
X.shape

(4, 17)

### 4개의 문장에 전체 말뭉치의 단어가 17개이다

In [24]:
num_samples, num_features = X.shape

### 확인

In [25]:
vectorizer.get_feature_names_out()

array(['괜찮아', '너무', '되기', '받은', '불편해', '사는', '사람', '상처', '쉬워', '아는',
       '아니야', '아무', '아이', '일도', '일찍', '좋은', '커버'], dtype=object)

### 행렬 읽는 법
- 첫번째 열의 숫자들은 위의 feature_name이 각각 몇번 나왔는지를 의미
- 예를들어 첫번째 문장에서 너무가 1번 받은이 1번 커버가 1번 나왔다는 것을 의미한다

In [26]:
X.toarray().transpose()

array([[0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 2, 0],
       [1, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0]])

### 테스트용 문장

In [27]:
new_post = ['상처받기 싫어 괜찮아']
new_post_tokens = []

for row in new_post:
    new_post_tokens.append(t.morphs(row))

new_post_for_vectorize = []

for content in new_post_tokens:
    sentence =''
    for word in content:
        sentence = sentence +' ' +word

    new_post_for_vectorize.append(sentence)

new_post_for_vectorize

[' 상처 받기 싫어 괜찮아']

### 벡터로 표현
- fit_transform과 transform의 차이
    - fit_transform은 처음 할 때
    - transform은 이미 벡터화 시킨 놈과 똑같은 모습으로 변형시킬때

In [28]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)
new_post_vec

<1x17 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [29]:
new_post_vec.toarray()

array([[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

### 유클리드 거리
- 벡터를 만들었으니 거리를 구할 수 있다

In [30]:
import scipy as sp

def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

### 4번째 문장이 테스트 문장과 제일 유사하다는 출력을 확인할 수 있다.

In [32]:
dist = []
for each in X:
    dist.append(dist_raw(each, new_post_vec))

dist

[2.449489742783178, 2.23606797749979, 3.1622776601683795, 2.0]

In [33]:
print("best post is ", dist.index(min(dist)), ", dist = ", min(dist))
print("test post is   ", new_post)
print("best post is   ", contents[dist.index(min(dist))])

best post is  3 , dist =  2.0
test post is    ['상처받기 싫어 괜찮아']
best post is    아무 일도 아니야 괜찮아


### 유사도 벡터 사이의 거리

In [34]:
for i in range(0, len(contents)):
    print(X.getrow(i).toarray())


print ("---------------")
print(new_post_vec.toarray())

[[0 1 0 1 0 0 0 1 0 0 0 0 1 0 1 0 1]]
[[0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0]]
[[0 0 1 0 0 1 2 0 1 0 0 0 0 0 0 1 0]]
[[1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0]]
---------------
[[1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]]


## TF_IDF